### Outputting Grid Metrics as 2D and 3D Arrays
*PB 10/18/22*

Based on OutputTifsandNetCDFs_3.ipy in Structural Complexity Mpala Project


In [2]:
import sys
sys.path.append('/n/home02/pbb/scripts/halo-metadata-server/Selenkay/bin/')
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import rasterio as rio
import xarray as xr
import rioxarray as rxr
from pathlib import Path
import geopandas as gpd
import time
from Functions import fill2Darray, classifyVeg_GGW, classifyVeg_GGST

### TBD: THIS HASN"T YET BEEN ADAPTED FOR THE MANGO RUN!
# Needs to be made to work in a loop and with the new metrics (cover and percentiles of the grass layer)
# 2/9/23
# only works for the initial run, not for "mango" yet

In [3]:
# Note: This currently works! 
# It is only running for outputs from a single shapefile though,
# In future versions, you may want to put it all in a loop 
# and change up the output folder structure (/Raster/30mRadius/Spot7B/2D/)
# Also, there is a noticable offset between the rasters and the shapefile
# the rasters appear shifted ~xysize/2 meters to the left and north
# PB 10/18/2022

# # # USER INPUTS: 

# Set a Label for this run (unique label id)
# label= 'initial'
# mango run - 2/8/23
label = 'mango'

# Radii to loop over
# radii = [20, 30, 50, 80, 130]

# Radius of shapefile to use
radius = 30

# Set the pickle metrics to draw from
metricdir = Path(f'/n/davies_lab/Users/pbb/SelenkayDiversity/data/out/Metrics/{label}/{radius}mRadius/')

# Horizontal Grid size
xysize = 0.5

# Ground threshold
groundthreshold = 0.05

# Set CRS
# Selenkay is 32737 (WGS84 UTM37S)
epsg='32737'

# Shapefile of Plots (for iterively setting boundaries of the grid)
# NOTE: This is the dissolved file without a buffer
# each feature is a plot polygon, marked by Site and Block number
shpf = Path(f'/n/home02/pbb/scripts/halo-metadata-server/Selenkay/data/in/BoundaryShapefiles/SelenkaySpotPolygons_IncreasingRadius/SelenkaySpotPolygons_{radius}mRadius.shp')
shpdf = gpd.read_file(shpf)

featureIDcol = 'Spot'

# Output Data:

# Outdirectory for rasters and netcdfs
outdir_rast = Path(f'/n/davies_lab/Users/pbb/SelenkayDiversity/data/out/Rasters/{label}/{radius}mRadius/')

if not outdir_rast.exists():
    outdir_rast.mkdir()

# list of 3D metrics to output,
# corresponding to the names in the saved cover dictionaries
# Note: all percentile metrics get output by default
metriclabels3d = ['Npulses', 'CoverD1', 'CoverD2', 'CoverD1byH', 'CoverD2byH', 'FHPD1', 'FHPD2']

# # #  END USER INPUTS

In [3]:
# For Testing- use only 7 and 8 here
# Sites = [s for s in shpdf[featureIDcol] if (('7' in s)|('8' in s))]
# Testing - Only use 3a
# shpdf = shpdf.query("Spot == '3a'").copy(deep=True)

# Sites to loop through
Sites = [s for s in shpdf[featureIDcol]]

# For each site
for s in Sites:
    
    print(f'Starting {featureIDcol} {s}...')
          
    start = time.time()
    
    # Get organized:
            
    # Make output dir if not already made
    # Note: removed the "2D" and "3D" product folder structure on 102722
    # Since it could get confusing with the Spot names
    if not Path(f'{outdir_rast}/{s}/').exists():
        Path(f'{outdir_rast}/{s}/').mkdir()
        
    # # Make 3D outdir if not already made
    # if not Path(f'{outdir_rast}/{s}/3D').exists():
    #     Path(f'{outdir_rast}/{s}/3D').mkdir()

    # 1) Load Inputs, and Redefine Grid

    # Load percentile height metric dict
    with open(f'{metricdir}/{s}_{xysize}mgrid_percmetrics.obj', 'rb') as of:
        perc = pickle.load(of)

    # Load Cover percentile dict
    with open(f'{metricdir}/{s}_{xysize}mgrid_covermetrics.obj', 'rb') as of:
        cover = pickle.load(of)
        
    # Load Complexity dict
    with open(f'{metricdir}/{s}_{xysize}mgrid_complexitymetrics.obj', 'rb') as of                                                                                                                                                                                                                                                                                                                                    :
        complexity = pickle.load(of)

    # Match site to get the plot polygon
    feat_gs = shpdf.loc[shpdf[featureIDcol]==s]

    # Set bounds of grid to fill
    xmin=float(feat_gs.geometry.bounds.minx)
    ymin=float(feat_gs.geometry.bounds.miny)
    xmax=float(feat_gs.geometry.bounds.maxx)
    ymax=float(feat_gs.geometry.bounds.maxy)

    # Make the coordinates of the grid
    x_grid = np.arange(xmin, xmax, step=xysize)
    y_grid = np.arange(ymin, ymax, step=xysize)

    # Mesh the grid into 2 matrices of x and y coordinates
    x_mesh, y_mesh = np.meshgrid(x_grid, y_grid)

    # Find the index of where all the data values belong in the grid
    xlist = [k[0] for k in cover.keys()]
    ylist = [k[1] for k in cover.keys()]

    xidx = []
    yidx = []

    # for each unique x,y location in the list of data values
    for x, y in zip(xlist, ylist):

        # Find it's unique x and y index location on the grid (ex: [0,1], [100,3], ...)        
        # xi = np.flatnonzero(x_grid==x)
        # yi = np.flatnonzero(y_grid==y)
        
        # Note: implemented the below, because I think there's a rounding error issue
        # grid x_grid and xlist don't quite match, off by about 0.009 m
        # 10/18/22
        
        # Find the closest value on the grid to make the index
        # calculate the difference array
        xdiff = np.absolute(x-x_grid)
        # find the index of minimum element from the array
        xindex = xdiff.argmin()
        
        # calculate the difference array
        ydiff = np.absolute(y-y_grid)
        # find the index of minimum element from the array
        yindex = ydiff.argmin()
        
        xi = np.flatnonzero(x_grid==x_grid[xindex])
        yi = np.flatnonzero(y_grid==y_grid[yindex])

        # If there is a location (not empty)
        if ((xi.size > 0) & (yi.size > 0)):

            # Add to the list
            xidx.append(xi[0])
            yidx.append(yi[0])

        else: 

            # Otherwise, mark them both with nans
            xidx.append(np.nan)
            yidx.append(np.nan)
    
    # Make an index for filtering arrays 
    # filterindex = np.array(xidx) >= 0
    
    # 2) Load, Reshape, and Output 2D Metrics

    # Get all the data dicts from the nested perc dictionary
    v = list(perc.values())

    # unpack all vars into lists
    perc0 = [i[0][0] for i in v]
    perc25 = [i[25][0] for i in v]
    perc50 = [i[50][0] for i in v]
    perc75 = [i[75][0] for i in v]
    perc98 = [i[98][0] for i in v]
    perc100 = [i[100][0] for i in v]
    meanh = [i['mean'][0] for i in v]
    stdh = [i['std'][0] for i in v]
    
    # Classify Vegetation using 98th percentile height
    vegtypeGGST = [classifyVeg_GGST(p, groundthreshold=groundthreshold) for p in perc98]
    vegtypeGGW = [classifyVeg_GGW(p, groundthreshold=groundthreshold) for p in perc98]
    
    # Get all the data vals from the nested complexity dictionary
    vc = list(complexity.values())

    # unpack all vars into lists
    nlayers = [i['nlayers'] for i in vc]
    maxpeakh = [i['maxpeakh'] for i in vc]
    gapsize = [i['gapsize'] for i in vc]
    ptoh = [i['ptoh'] for i in vc]
    cscore = [i['cscore'] for i in vc]
    FHD = [i['FHD'] for i in vc]
    VDR = [i['VDR'] for i in vc]
    meanpeakh = [i['meanpeakh'] for i in vc]
    stdpeakh = [i['stdpeakh'] for i in vc]
    cvpeakh = [i['cvpeakh'] for i in vc]
    VDRpeak = [i['VDRpeak'] for i in vc]
    herbh = [i['herbh'] for i in vc]

    # 'nlayers', 'gapsize', 'maxpeakh','ptoh', 'cscore', 'FHD', 'VDR', 'meanpeakh', 'stdpeakh', 'cvpeakh'
    # nlayers, gapsize, maxpeakh, ptoh, cscore, FHD, VDR, meanpeakh, stdpeakh, cvpeakh

    # Reshape vars into 2D arrays and store in a dictionary
    outdict = {}.fromkeys(['0th','25th', '50th', '75th', '98th', '100th', 'Mean', 'Std', 'vegtypeGGST', 'vegtypeGGW',
                           'nlayers', 'gapsize', 'maxpeakh','ptoh', 'cscore', 'FHD', 'VDR', 'meanpeakh', 'stdpeakh', 'cvpeakh', 'VDRpeak', 'herbh'])

    for m, l in zip([perc0, perc25, perc50, perc75, perc98, perc100, meanh, stdh, vegtypeGGST, vegtypeGGW,
                     nlayers, gapsize,maxpeakh, ptoh, cscore, FHD, VDR, meanpeakh, stdpeakh, cvpeakh, VDRpeak, herbh],
                    ['0th','25th', '50th', '75th', '98th', '100th', 'Mean', 'Std', 'vegtypeGGST', 'vegtypeGGW',
                     'nlayers', 'gapsize', 'maxpeakh','ptoh', 'cscore', 'FHD', 'VDR', 'meanpeakh', 'stdpeakh', 'cvpeakh', 'VDRpeak', 'herbh']):

        outdict[l] = fill2Darray(data=m, shape=x_mesh.shape,
                                 xindices=xidx, yindices=yidx, plot=False)

    # Output 2D Arrays as geotifs
    for l in ['0th','25th', '50th', '75th', '98th', '100th', 'Mean', 'Std', 'vegtypeGGST', 'vegtypeGGW',
              'nlayers', 'gapsize', 'maxpeakh','ptoh', 'cscore', 'FHD', 'VDR', 'meanpeakh', 'stdpeakh', 'cvpeakh', 'VDRpeak', 'herbh']:

        # Grab one metric for output
        m = outdict[l]

        # Mirror Image the metric
        # Note: You do this because rioxarray wants data
        # ordered with positive x (left to right)
        # but with negative y (top to bottom)
        # This really just makes the export to geotif go smoothly,
        # with a correct affine transform 
        m = np.flipud(m)

        # Also flip y coordinates, so that they're correctly read into x array
        # y_grid_flip = np.flip(y_grid)
        # NOTE: adding xysize to y coord so that it marks Top left corner
        # instead of bottom left - this is in accordance with raster data and rioxarray
        y_grid_flip = np.flip(y_grid) + xysize

        # put in an xarray
        # 2D
        m_xr = xr.DataArray(data=m,
                            coords={"y": y_grid_flip,
                                    "x": x_grid},
                            dims=["y", "x"])

        # Write CRS and Nodata value and dims to xarray
        m_xr.rio.write_crs(f"epsg:{epsg}",
                           inplace=True)
        m_xr.rio.write_nodata(-9999,
                              inplace=True)
        m_xr.rio.set_spatial_dims(x_dim="x",
                                  y_dim="y",
                                  inplace=True)
        m_xr.rio.write_coordinate_system(inplace=True)

        # make an output Label
        label = l.replace(' ', '').replace('.', 'p').replace('[m]', '')

        m_xr.rio.to_raster(f'{outdir_rast}/{s}/{s}_{label}.tif')

        # print(f'{l} done')

    end2d = time.time()

    print(f'\tFinished 2D outputs for {featureIDcol} {s}. {end2d-start} seconds.')

    
    # 3) Load, Reshape, and Output 3D Metrics

    # Unpack cover values as a list
    v3 = list(cover.values())

    # Get list of heights for later
    hbins = v3[0]['HeightBins']
    
    # Make shape for output array
    # note - y, then x here
    shape = (len(y_grid), len(x_grid))

    # for each set of 3D metrics
    for l3 in metriclabels3d:

        # list to fill with xarrays from each height
        xr_list = []

        # Deals with an issue caused by using np.diff
        # if there's a difference metric, then the last height does not have a value
        if (("byH" in l3) | ('FHP' in l3)) :
            hbinz = hbins[1:]
        else:
            hbinz = hbins

        # for each height bin
        for hidx, h in enumerate(hbinz): 

            # make an array of all the values
            m = np.array([i[l3][hidx] for i in v3])

            # make an empty output array, filled with nans
            output_array = np.full(shape, np.nan)

            # if that data needs to be filtered
#             if filterindex:

#                 data = m[filterindex]
#                 xindices = np.array(xidx)[filterindex]
#                 yindices = np.array(yidx)[filterindex]

            # Convert to int
            xindices = np.array(xidx).astype(int)
            yindices = np.array(yidx).astype(int)

            # Fill any inf values in data with -9999
            # data[np.isfinite(data, where=False)] = -9999

            # stick into the array
            # IMPORTANT NOTE HERE: it's y, then x, not the other way around
            output_array[yindices, xindices] = m

            # Mirror Image the metric
            # Note: You do this because rioxarray wants data
            # ordered with positive x (left to right)
            # but with negative y (top to bottom)
            # This really just makes the export to geotif go smoothly,
            # with a correct affine transform 
            output_array = np.flipud(output_array)

            # Also flip y coordinates, so that they're correctly read into x array 
            # y_grid_flip = np.flip(y_grid)
            # NOTE: adding xysize to y coord so that it marks Top left corner
            # instead of bottom left - this is in accordance with raster data and rioxarray
            y_grid_flip = np.flip(y_grid) + xysize

            # put in an xarray
            # 2D
            m_xr = xr.DataArray(data=output_array,
                                coords={"y": y_grid_flip,
                                        "x": x_grid},
                                dims=["y", "x"])

            # Write CRS and Nodata value
            m_xr.rio.write_crs(f"epsg:{epsg}", inplace=True)
            m_xr.rio.write_nodata(-9999, inplace=True)
            m_xr.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
            m_xr.rio.write_coordinate_system(inplace=True)

            # add to list 
            xr_list.append(m_xr)

        # Concatenate dataarrays
        # https://docs.xarray.dev/en/stable/user-guide/combining.html#combine
        # setting the values of the new dimension to be height
        ds = xr.concat(xr_list, hbinz)
        ds = ds.rename({'concat_dim':'z'})

        # Set it's name
        ds.name = l3

        # output to netcdf
        ds.to_netcdf(f'{outdir_rast}/{s}/{s}_{l3}.nc')

    end3d = time.time()

    print(f'\tFinished 3D outputs for {featureIDcol} {s}. {end3d-start} seconds.\n')
   

0th done
25th done
50th done
75th done
98th done
100th done
Mean done
Std done
vegtypeGGST done
vegtypeGGW done
nlayers done
gapsize done
maxpeakh done
ptoh done
cscore done
FHD done
VDR done
meanpeakh done
stdpeakh done
cvpeakh done
VDRpeak done
herbh done
Finished 2D outputs for Spot 1E. 2.330183506011963 seconds. 

Finished 3D outputs for Spot 1E. 15.284670114517212 seconds.

0th done
25th done
50th done
75th done
98th done
100th done
Mean done
Std done
vegtypeGGST done
vegtypeGGW done
nlayers done
gapsize done
maxpeakh done
ptoh done
cscore done
FHD done
VDR done
meanpeakh done
stdpeakh done
cvpeakh done
VDRpeak done
herbh done
Finished 2D outputs for Spot 1D. 2.395358085632324 seconds. 

Finished 3D outputs for Spot 1D. 15.692286252975464 seconds.

0th done
25th done
50th done
75th done
98th done
100th done
Mean done
Std done
vegtypeGGST done
vegtypeGGW done
nlayers done
gapsize done
maxpeakh done
ptoh done
cscore done
FHD done
VDR done
meanpeakh done
stdpeakh done
cvpeakh done
VD

In [4]:
# Figure showing slight offset between coords in xlist (from cover_dict) and coords made with np.arange
# fig, ax = plt.subplots()
# ax.scatter(x_mesh, y_mesh, s=2, c='b')
# ax.scatter(xlist, ylist, s=2, c='k')
# ax.axis('equal')
# ax.margins(x=-0.49, y=-0.49) 

In [5]:
# plt.hist(np.array(vegtypeGGST)[np.array(vegtypeGGST)>0])